# __Migrating One-Stop Superstore CSV Data to SQL with Python__

<div style="text-align: center;">  
   <img src="https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-1574692455.jpg?c=16x9&q=h_653,w_1160,c_fill/f_webp" alt="Superstore" style="width: 600px; height: 400px;" title="A superstore" />  
</div>

<span style="font-size: 20px;"> __Problem Statement__ </span>

<span style="font-size: 18px;">One Stop Superstore has a large amount of data stored in XLSX files. This format is inefficient for complex queries and data analysis. The goal is to migrate this data into a SQL database for improved performance and data management capabilities. </span>

<span style="font-size: 20px;">__Solution__ </span>

<span style="font-size: 18px;">Utilize Python's powerful data manipulation and database interaction libraries to: </span>

<span style="font-size: 18px;">
    
1. **Import CSV Data:**
   - Read the CSV files using the `csv` or `pandas` library.
   - Handle potential inconsistencies in data formats (e.g., missing values, incorrect data types).

2. **Data Cleaning:**
   - Identify and remove duplicate records.
   - Correct inconsistencies in data values (e.g., typos, incorrect date formats).
   - Handle missing values by imputation or deletion.
   - Normalize data to ensure consistency (e.g., convert all date formats to a standard).

3. **Create SQL Database:**
   - Connect to a SQL Server.
   - Create necessary tables with appropriate data types and constraints based on the CSV data structure.

4. **Insert Data into SQL:**
   - Iterate over the cleaned CSV data and insert each row into the corresponding SQL table.
   - Use parameterized queries to prevent SQL injection vulnerabilities.

5. **Data Validation:**
   - Verify the integrity of the migrated data by running SQL queries to check for consistency and accuracy.
</span>

## Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [2]:
def get_excel_sheet_names(file_path):
    """
    Returns a list of sheet names in an Excel file.

    Parameters:
        file_path (str): The path to the Excel file.

    Returns:
        list: A list of sheet names in the Excel file.
    """

    # Read the Excel file using pandas
    excel_file = pd.ExcelFile(file_path)

    # Get the sheet names and remove unprintable characters
    sheet_names = excel_file.sheet_names

    return sheet_names

In [3]:
file_path = 'norm_trans.xlsx'
sheet_names = get_excel_sheet_names(file_path)  
print(sheet_names)

['Customer', 'Store', 'Product', 'Product Subcategory', 'Product Category', 'Order', 'Order Item']


In [161]:
# Read all sheets into separate DataFrames with custom names
dataframes = {}
for sheet_name in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    dataframes[sheet_name] = df

# Access DataFrames separately
customer_df = dataframes["Customer"]
store_df = dataframes["Store"]
product_df = dataframes["Product"]
product_subcategory_df = dataframes["Product Subcategory"]
product_category_df = dataframes["Product Category"]
order_df = dataframes["Order"]
order_item_df = dataframes["Order Item"]

## EDA and Cleaning

In [162]:
# customer_df

In [163]:
customer_df.head()

,CustomerID,CustomerGender,CustomerName,CustomerCity,CustomerStateCode,CustomerState,CustomerZip,CustomerCountry,CustomerContinent,CustomerDOB
0,265598,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,1269051,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,266019,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,1107461,Male,Sam Nelson,BURSCOUGH BRIDGE,West Lancashire,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,844003,Male,Ouassim Heuver,Leeuwarden,FR,Friesland,8916 CD,Netherlands,Europe,1995-06-22


In [164]:
customer_df.dtypes

CustomerID                    int64
CustomerGender               object
CustomerName                 object
CustomerCity                 object
CustomerStateCode            object
CustomerState                object
CustomerZip                  object
CustomerCountry              object
CustomerContinent            object
CustomerDOB          datetime64[ns]
dtype: object

In [165]:
print ("The number of rows are:", customer_df.shape[0])
print ("The number of columns are:", customer_df.shape[1])

The number of rows are: 11887
The number of columns are: 10


In [166]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11887 entries, 0 to 11886
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CustomerID         11887 non-null  int64         
 1   CustomerGender     11887 non-null  object        
 2   CustomerName       11887 non-null  object        
 3   CustomerCity       11887 non-null  object        
 4   CustomerStateCode  11878 non-null  object        
 5   CustomerState      11887 non-null  object        
 6   CustomerZip        11887 non-null  object        
 7   CustomerCountry    11887 non-null  object        
 8   CustomerContinent  11887 non-null  object        
 9   CustomerDOB        11887 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 928.8+ KB


In [167]:
customer_df.describe()

,CustomerID,CustomerDOB
count,1.188700e+04,11887
mean,1.110392e+06,1968-10-22 13:40:29.174728696
min,3.010000e+02,1935-02-04 00:00:00
25%,5.749655e+05,1952-03-09 12:00:00
50%,1.153522e+06,1968-09-28 00:00:00
75%,1.627821e+06,1985-07-24 00:00:00
max,2.099937e+06,2002-01-30 00:00:00
std,5.991078e+05,NaN


In [168]:
customer_df.isna().sum()

CustomerID           0
CustomerGender       0
CustomerName         0
CustomerCity         0
CustomerStateCode    9
CustomerState        0
CustomerZip          0
CustomerCountry      0
CustomerContinent    0
CustomerDOB          0
dtype: int64

In [169]:
customer_df[customer_df.duplicated()]

,CustomerID,CustomerGender,CustomerName,CustomerCity,CustomerStateCode,CustomerState,CustomerZip,CustomerCountry,CustomerContinent,CustomerDOB


In [170]:
print(f'Features: {customer_df.columns.tolist()}')
print(f'Missing Values: {customer_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{customer_df.nunique()}')

Features: ['CustomerID', 'CustomerGender', 'CustomerName', 'CustomerCity', 'CustomerStateCode', 'CustomerState', 'CustomerZip', 'CustomerCountry', 'CustomerContinent', 'CustomerDOB']
Missing Values: 9
Unique Values: 

CustomerID           11887
CustomerGender           2
CustomerName         11783
CustomerCity          6570
CustomerStateCode      448
CustomerState          491
CustomerZip           7771
CustomerCountry          8
CustomerContinent        3
CustomerDOB           9382
dtype: int64


In [171]:
# store_df

In [172]:
store_df.head()

,StoreID,StoreCountry,StoreState,StoreSqMeters,StoreOpenDate
0,1,Online,Online,0,2010-01-01
1,2,Canada,Nunavut,1210,2015-04-04
2,3,United Kingdom,Belfast,1800,2015-04-04
3,4,Netherlands,Friesland,1540,2015-12-09
4,5,United States,Alaska,1190,2015-01-01


In [173]:
store_df.dtypes

StoreID                   int64
StoreCountry             object
StoreState               object
StoreSqMeters             int64
StoreOpenDate    datetime64[ns]
dtype: object

In [174]:
print ("The number of rows are:", store_df.shape[0])
print ("The number of columns are:", store_df.shape[1])

The number of rows are: 58
The number of columns are: 5


In [175]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   StoreID        58 non-null     int64         
 1   StoreCountry   58 non-null     object        
 2   StoreState     58 non-null     object        
 3   StoreSqMeters  58 non-null     int64         
 4   StoreOpenDate  58 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 2.4+ KB


In [176]:
store_df.describe()

,StoreID,StoreSqMeters,StoreOpenDate
count,58.000000,58.000000,58
mean,29.500000,1375.689655,2011-11-11 21:31:02.068965632
min,1.000000,0.000000,2005-03-04 00:00:00
25%,15.250000,1021.250000,2009-12-15 00:00:00
50%,29.500000,1365.000000,2012-03-22 12:00:00
75%,43.750000,2000.000000,2014-05-17 12:00:00
max,58.000000,2105.000000,2019-03-05 00:00:00
std,16.886879,627.319170,NaN


In [177]:
store_df[store_df.duplicated()]

,StoreID,StoreCountry,StoreState,StoreSqMeters,StoreOpenDate


In [178]:
print(f'Features: {store_df.columns.tolist()}')
print(f'Missing Values: {store_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{store_df.nunique()}')

Features: ['StoreID', 'StoreCountry', 'StoreState', 'StoreSqMeters', 'StoreOpenDate']
Missing Values: 0
Unique Values: 

StoreID          58
StoreCountry      9
StoreState       58
StoreSqMeters    33
StoreOpenDate    23
dtype: int64


In [179]:
# product_df

In [180]:
product_df.head()

,ProductID,ProductName,ProductBrand,ProductColor,ProductCost,ProductPrice,ProductSubcategoryID
0,1304,Contoso Lens Adapter M450 White,Contoso,White,31.27,68.00,406
1,1048,A. Datum SLR Camera X136 Silver,A. Datum,Silver,141.47,427.00,402
2,2007,Fabrikam Microwave 1.5CuFt X1100 Black,Fabrikam,Black,220.64,665.94,803
3,1106,Contoso SLR Camera M146 Orange,Contoso,Orange,148.08,322.00,402
4,373,Adventure Works Laptop8.9 E0890 White,Adventure Works,White,166.20,326.00,301


In [181]:
product_df.dtypes

ProductID                 int64
ProductName              object
ProductBrand             object
ProductColor             object
ProductCost             float64
ProductPrice            float64
ProductSubcategoryID      int64
dtype: object

In [182]:
print ("The number of rows are:", product_df.shape[0])
print ("The number of columns are:", product_df.shape[1])

The number of rows are: 2492
The number of columns are: 7


In [183]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2492 entries, 0 to 2491
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductID             2492 non-null   int64  
 1   ProductName           2492 non-null   object 
 2   ProductBrand          2492 non-null   object 
 3   ProductColor          2492 non-null   object 
 4   ProductCost           2492 non-null   float64
 5   ProductPrice          2492 non-null   float64
 6   ProductSubcategoryID  2492 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 136.4+ KB


In [184]:
product_df.describe()

,ProductID,ProductCost,ProductPrice,ProductSubcategoryID
count,2492.000000,2492.000000,2492.000000,2492.000000
mean,1249.902087,147.875405,357.571120,488.902087
std,723.560119,184.637974,495.398809,228.881411
min,1.000000,0.480000,0.950000,101.000000
25%,624.750000,32.250000,68.750000,305.000000
50%,1247.500000,86.670000,199.990000,406.000000
75%,1870.250000,186.855000,410.250000,801.000000
max,2517.000000,1060.220000,3199.990000,808.000000


In [185]:
product_df.isna().sum()

ProductID               0
ProductName             0
ProductBrand            0
ProductColor            0
ProductCost             0
ProductPrice            0
ProductSubcategoryID    0
dtype: int64

In [186]:
product_df[product_df.duplicated()]

,ProductID,ProductName,ProductBrand,ProductColor,ProductCost,ProductPrice,ProductSubcategoryID


In [187]:
print(f'Features: {product_df.columns.tolist()}')
print(f'Missing Values: {product_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{product_df.nunique()}')

Features: ['ProductID', 'ProductName', 'ProductBrand', 'ProductColor', 'ProductCost', 'ProductPrice', 'ProductSubcategoryID']
Missing Values: 0
Unique Values: 

ProductID               2492
ProductName             2492
ProductBrand              11
ProductColor              16
ProductCost              480
ProductPrice             426
ProductSubcategoryID      32
dtype: int64


In [188]:
# product_subcategory_df

In [189]:
product_subcategory_df.head()

,ProductSubcategoryID,ProductSubcategory,ProductCategoryID
0,101,MP4&MP3,1
1,104,Recording Pen,1
2,106,Bluetooth Headphones,1
3,201,Televisions,2
4,202,VCD & DVD,2


In [190]:
product_subcategory_df.dtypes

ProductSubcategoryID     int64
ProductSubcategory      object
ProductCategoryID        int64
dtype: object

In [191]:
print ("The number of rows are:", product_subcategory_df.shape[0])
print ("The number of columns are:", product_subcategory_df.shape[1])

The number of rows are: 32
The number of columns are: 3


In [192]:
product_subcategory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ProductSubcategoryID  32 non-null     int64 
 1   ProductSubcategory    32 non-null     object
 2   ProductCategoryID     32 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 900.0+ bytes


In [193]:
product_subcategory_df.describe()

,ProductSubcategoryID,ProductCategoryID
count,32.00000,32.000000
mean,469.28125,4.656250
std,247.21523,2.470593
min,101.00000,1.000000
25%,302.50000,3.000000
50%,405.50000,4.000000
75%,726.75000,7.250000
max,808.00000,8.000000


In [194]:
product_subcategory_df.isna().sum()

ProductSubcategoryID    0
ProductSubcategory      0
ProductCategoryID       0
dtype: int64

In [195]:
product_subcategory_df[product_subcategory_df.duplicated()]

,ProductSubcategoryID,ProductSubcategory,ProductCategoryID


In [196]:
print(f'Features: {product_subcategory_df.columns.tolist()}')
print(f'Missing Values: {product_subcategory_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{product_subcategory_df.nunique()}')

Features: ['ProductSubcategoryID', 'ProductSubcategory', 'ProductCategoryID']
Missing Values: 0
Unique Values: 

ProductSubcategoryID    32
ProductSubcategory      32
ProductCategoryID        8
dtype: int64


In [197]:
# product_category_df

In [198]:
product_category_df

,ProductCategoryID,ProductCategory
0,1,Audio
1,2,TV and Video
2,3,Computers
3,4,Cameras and camcorders
4,5,Cell phones
5,6,"Music, Movies and Audio Books"
6,7,Games and Toys
7,8,Home Appliances


In [199]:
product_category_df.dtypes

ProductCategoryID     int64
ProductCategory      object
dtype: object

In [200]:
print ("The number of rows are:", product_category_df.shape[0])
print ("The number of columns are:", product_category_df.shape[1])

The number of rows are: 8
The number of columns are: 2


In [201]:
product_category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ProductCategoryID  8 non-null      int64 
 1   ProductCategory    8 non-null      object
dtypes: int64(1), object(1)
memory usage: 260.0+ bytes


In [202]:
product_category_df.describe()

,ProductCategoryID
count,8.00000
mean,4.50000
std,2.44949
min,1.00000
25%,2.75000
50%,4.50000
75%,6.25000
max,8.00000


In [203]:
product_category_df[product_category_df.duplicated()]

,ProductCategoryID,ProductCategory


In [204]:
print(f'Features: {product_category_df.columns.tolist()}')
print(f'Missing Values: {product_category_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{product_category_df.nunique()}')

Features: ['ProductCategoryID', 'ProductCategory']
Missing Values: 0
Unique Values: 

ProductCategoryID    8
ProductCategory      8
dtype: int64


In [205]:
# order_df

In [206]:
order_df.head()

,OrderID,CustomerID,StoreID,OrderDate,DeliveryDate
0,366000,265598,10,2016-01-01,NaT
1,366001,1269051,1,2016-01-01,2016-01-13
2,366002,266019,1,2016-01-01,2016-01-12
3,366004,1107461,38,2016-01-01,NaT
4,366005,844003,33,2016-01-01,NaT


In [207]:
order_df.dtypes

OrderID                  int64
CustomerID               int64
StoreID                  int64
OrderDate       datetime64[ns]
DeliveryDate    datetime64[ns]
dtype: object

In [208]:
print ("The number of rows are:", order_df.shape[0])
print ("The number of columns are:", order_df.shape[1])

The number of rows are: 26326
The number of columns are: 5


In [209]:
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26326 entries, 0 to 26325
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   OrderID       26326 non-null  int64         
 1   CustomerID    26326 non-null  int64         
 2   StoreID       26326 non-null  int64         
 3   OrderDate     26326 non-null  datetime64[ns]
 4   DeliveryDate  5580 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(3)
memory usage: 1.0 MB


In [210]:
order_df.describe()

,OrderID,CustomerID,StoreID,OrderDate,DeliveryDate
count,2.632600e+04,2.632600e+04,26326.000000,26326,5580
mean,1.433195e+06,1.183849e+06,32.032895,2018-12-03 04:19:22.895996416,2019-01-21 13:06:50.322580736
min,3.660000e+05,3.010000e+02,1.000000,2016-01-01 00:00:00,2016-01-06 00:00:00
25%,1.127014e+06,6.954330e+05,8.000000,2018-01-31 00:00:00,2018-05-18 00:00:00
50%,1.499034e+06,1.265357e+06,37.000000,2019-02-07 00:00:00,2019-03-11 00:00:00
75%,1.788025e+06,1.682062e+06,53.000000,2019-11-23 00:00:00,2019-12-19 00:00:00
max,2.243032e+06,2.099937e+06,66.000000,2021-02-20 00:00:00,2021-02-27 00:00:00
std,4.525186e+05,5.832701e+05,22.752358,NaN,NaN


In [211]:
order_df.isna().sum()

OrderID             0
CustomerID          0
StoreID             0
OrderDate           0
DeliveryDate    20746
dtype: int64

In [212]:
order_df[order_df.duplicated()]

,OrderID,CustomerID,StoreID,OrderDate,DeliveryDate


In [213]:
print(f'Features: {order_df.columns.tolist()}')
print(f'Missing Values: {order_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{order_df.nunique()}')

Features: ['OrderID', 'CustomerID', 'StoreID', 'OrderDate', 'DeliveryDate']
Missing Values: 20746
Unique Values: 

OrderID         26326
CustomerID      11887
StoreID            57
OrderDate        1641
DeliveryDate     1492
dtype: int64


In [214]:
# order_item_df

In [215]:
order_item_df.head()

,OrderNumber,LineItem,ProductID,Quantity,ProductCost,ProductPrice
0,366000,1,1304,1,31.27,68.00
1,366001,1,1048,2,141.47,427.00
2,366001,2,2007,1,220.64,665.94
3,366002,1,1106,7,148.08,322.00
4,366002,2,373,1,166.20,326.00


In [216]:
order_item_df.dtypes

OrderNumber       int64
LineItem          int64
ProductID         int64
Quantity          int64
ProductCost     float64
ProductPrice    float64
dtype: object

In [217]:
print ("The number of rows are:", order_item_df.shape[0])
print ("The number of columns are:", order_item_df.shape[1])

The number of rows are: 62884
The number of columns are: 6


In [218]:
order_item_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62884 entries, 0 to 62883
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OrderNumber   62884 non-null  int64  
 1   LineItem      62884 non-null  int64  
 2   ProductID     62884 non-null  int64  
 3   Quantity      62884 non-null  int64  
 4   ProductCost   62884 non-null  float64
 5   ProductPrice  62884 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 2.9 MB


In [219]:
order_item_df.describe()

,OrderNumber,LineItem,ProductID,Quantity,ProductCost,ProductPrice
count,6.288400e+04,62884.000000,62884.000000,62884.000000,62884.000000,62884.000000
mean,1.430905e+06,2.164207,1125.859344,3.144790,116.253013,280.569848
std,4.532963e+05,1.365170,709.244010,2.256371,140.033327,382.827515
min,3.660000e+05,1.000000,1.000000,1.000000,0.480000,0.950000
25%,1.121017e+06,1.000000,437.000000,1.000000,22.050000,47.950000
50%,1.498016e+06,2.000000,1358.000000,2.000000,79.530000,181.000000
75%,1.788010e+06,3.000000,1650.000000,4.000000,138.420000,300.000000
max,2.243032e+06,7.000000,2517.000000,10.000000,1060.220000,3199.990000


In [220]:
order_item_df.isna().sum()

OrderNumber     0
LineItem        0
ProductID       0
Quantity        0
ProductCost     0
ProductPrice    0
dtype: int64

In [221]:
order_item_df[order_item_df.duplicated()]

,OrderNumber,LineItem,ProductID,Quantity,ProductCost,ProductPrice


In [222]:
print(f'Features: {order_item_df.columns.tolist()}')
print(f'Missing Values: {order_item_df.isnull().values.sum()}')
print(f'Unique Values: \n\n{order_item_df.nunique()}')

Features: ['OrderNumber', 'LineItem', 'ProductID', 'Quantity', 'ProductCost', 'ProductPrice']
Missing Values: 0
Unique Values: 

OrderNumber     26326
LineItem            7
ProductID        2492
Quantity           10
ProductCost       480
ProductPrice      426
dtype: int64


### Rename columns

In [223]:
order_item_df.columns

Index(['OrderNumber', 'LineItem', 'ProductID', 'Quantity', 'ProductCost',
       'ProductPrice'],
      dtype='object')

In [224]:
# Renaming columns
customer_df.rename(columns={'CustomerID': 'id', 'CustomerGender': 'gender', 'CustomerName': 'name',
                            'CustomerCity': 'city', 'CustomerStateCode' :'state_code', 'CustomerState': 'state',
                            'CustomerZip': 'zip', 'CustomerCountry': 'country', 'CustomerContinent': 'continent',
                            'CustomerDOB': 'dob'}, inplace=True)

In [225]:
# Renaming store columns
store_df.rename(columns={'StoreID': 'id', 'StoreCountry': 'country', 'StoreState': 'state',
                            'StoreSqMeters': 'sq_meters', 'StoreOpenDate': 'open_date'}, inplace=True)

In [226]:
# Renaming columns
product_df.rename(columns={'ProductID': 'id','ProductName': 'name', 'ProductBrand': 'brand',
                           'ProductCost': 'cost', 'ProductColor' : 'color', 'ProductPrice': 'price',
                           'ProductSubcategoryID' :'subcategory_id'}, inplace=True)

In [227]:
# Renaming columns
product_subcategory_df.rename(columns={'ProductSubcategoryID': 'id', 'ProductSubcategory': 'category',
                            'ProductCategoryID': 'category_id'}, inplace=True)

In [228]:
# Renaming columns
product_category_df.rename(columns={'ProductCategoryID': 'id', 'ProductCategory': 'category'}, inplace=True)

In [229]:
# Renaming columns
order_df.rename(columns={'OrderID': 'id', 'CustomerID': 'customer_id', 'StoreID': 'store_id',
                            'OrderDate': 'order_date', 'DeliveryDate': 'delivery_date'}, inplace=True)

In [230]:
# Renaming columns
order_item_df.rename(columns={'OrderNumber': 'order_id', 'LineItem': 'line_item', 'ProductID': 'product_id',
                            'Quantity': 'quantity', 'ProductCost' :'product_cost', 'ProductPrice': 'product_price'}
                     , inplace=True)

### Cleaning names

In [231]:
import re

# Function to clean names
def clean_name(name):
    # Remove leading and trailing spaces, underscores, hyphens, and slashes
    name = re.sub(r'^[\s\-_\/]+|[\s\-_\/]+$', '', name)
    # Remove numbers
    name = re.sub(r'\d+', '', name)
    # Replace any remaining underscores, hyphens, or slashes with a space
    name = re.sub(r'[\-_\/]', ' ', name)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

In [232]:
# Apply the function to the 'names' column
customer_df['name'] = customer_df['name'].apply(clean_name)
customer_df.head()

,id,gender,name,city,state_code,state,zip,country,continent,dob
0,265598,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,1269051,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,266019,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,1107461,Male,Sam Nelson,BURSCOUGH BRIDGE,West Lancashire,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,844003,Male,Ouassim Heuver,Leeuwarden,FR,Friesland,8916 CD,Netherlands,Europe,1995-06-22


In [233]:
# Making all zips into upper case

In [234]:
def convert_to_uppercase_with_numbers(df, column_name):
    """
    Converts a column to uppercase while preserving numbers.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        column_name (str): The name of the column to convert.

    Returns:
        The modified DataFrame.
    """

    # Use a regular expression to replace lowercase letters with uppercase
    df[column_name] = df[column_name].astype(str).str.replace(r'[a-z]', lambda x: x.group().upper(), regex=True)
    return df

In [235]:
customer_df = convert_to_uppercase_with_numbers(customer_df, 'zip')

In [236]:
customer_df.head()

,id,gender,name,city,state_code,state,zip,country,continent,dob
0,265598,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,1269051,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,266019,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,1107461,Male,Sam Nelson,BURSCOUGH BRIDGE,West Lancashire,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,844003,Male,Ouassim Heuver,Leeuwarden,FR,Friesland,8916 CD,Netherlands,Europe,1995-06-22


In [237]:
# Making other columns into title case

In [238]:
customer_df[['name', 'city', 'state', 'country', 'continent']] = customer_df[['name', 'city', 'state', 'country','continent']].apply(lambda x: x.str.title())

In [239]:
customer_df.head()

,id,gender,name,city,state_code,state,zip,country,continent,dob
0,265598,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,1269051,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,266019,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,1107461,Male,Sam Nelson,Burscough Bridge,West Lancashire,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,844003,Male,Ouassim Heuver,Leeuwarden,FR,Friesland,8916 CD,Netherlands,Europe,1995-06-22


### Cleaning redundant data

In [240]:
# Drop state_code from the customer_df

In [241]:
customer_df.drop(['state_code'], axis=1, inplace= True)

In [242]:
customer_df.head()

,id,gender,name,city,state,zip,country,continent,dob
0,265598,Male,Tyler Vaught,London,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,1269051,Male,Frank Upchurch,Auberry,California,93602,United States,North America,1995-11-20
2,266019,Female,Joan Roche,Red Deer,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,1107461,Male,Sam Nelson,Burscough Bridge,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,844003,Male,Ouassim Heuver,Leeuwarden,Friesland,8916 CD,Netherlands,Europe,1995-06-22


## Save as csv

In [243]:
order_item_df.head()

,order_id,line_item,product_id,quantity,product_cost,product_price
0,366000,1,1304,1,31.27,68.00
1,366001,1,1048,2,141.47,427.00
2,366001,2,2007,1,220.64,665.94
3,366002,1,1106,7,148.08,322.00
4,366002,2,373,1,166.20,326.00


In [244]:
customer_df.set_index(customer_df.columns[0], inplace=True)

In [245]:
customer_df.to_csv('customer_df.csv')

In [246]:
store_df.set_index(store_df.columns[0], inplace=True)
store_df.to_csv('store_df.csv')

In [247]:
product_df.set_index(product_df.columns[0], inplace=True)
product_df.to_csv('product_df.csv')

In [248]:
product_subcategory_df.set_index(product_subcategory_df.columns[0], inplace=True)
product_subcategory_df.to_csv('product_subcategory_df.csv')

In [249]:
product_category_df.set_index(product_category_df.columns[0], inplace=True)
product_category_df.to_csv('product_category_df.csv')

In [250]:
order_df.set_index(order_df.columns[0], inplace=True)
order_df.to_csv('order_df.csv')

In [251]:
order_item_df.set_index(order_item_df.columns[0], inplace=True)
order_item_df.to_csv('order_item_df.csv')

## Reading into SQL

In [108]:
from sqlalchemy import create_engine
import pyodbc
import sqlalchemy as sal

### Find potential primary keys

In [109]:
def find_potential_primary_key(df):
  """
  Finds potential primary key columns in a DataFrame.

  Args:
    df: The DataFrame to analyze.

  Returns:
    A list of column names that could be primary keys.
  """

  potential_primary_keys = []
  for column in df.columns:
    if df[column].is_unique:
      potential_primary_keys.append(column)

  # Print the potential primary keys
  for column in potential_primary_keys:
    print(f"Column '{column}' could be a primary key.")

In [110]:
find_potential_primary_key(customer_df)

In [111]:
find_potential_primary_key(store_df)

Column 'state' could be a primary key.


In [112]:
find_potential_primary_key(product_df)

Column 'name' could be a primary key.


In [113]:
find_potential_primary_key(product_subcategory_df)

Column 'category' could be a primary key.


In [114]:
find_potential_primary_key(product_category_df)

Column 'category' could be a primary key.


In [115]:
find_potential_primary_key(order_df)

In [116]:
find_potential_primary_key(order_item_df)

__Order Item has three composite primary key: order_id, line_item and product_id__

### Finding foreign keys

In [117]:
df_list = [customer_df, store_df, product_df, product_subcategory_df,
           product_category_df, order_df, order_item_df]

In [118]:
def find_foreign_keys(df1, df2):
    foreign_keys = []
    for col1 in df1.columns:
        for col2 in df2.columns:
            if df2[col2].isin(df1[col1]).all():
                foreign_keys.append((col2, col1))
    return foreign_keys

In [119]:
# Foreign keys for customer_df

In [120]:
dataframes = [store_df, product_df, product_subcategory_df,
           product_category_df, order_df, order_item_df]
cust_foreign_keys_dict = {}  
  
for df in dataframes:  
    foreign_keys = find_foreign_keys(customer_df, df)  
    cust_foreign_keys_dict[id(df)] = foreign_keys  
  
print(cust_foreign_keys_dict)

{2276207450720: [], 2276209177952: [], 2276226530432: [], 2276199232576: [], 2276235311104: [], 2276230857216: []}


__customer_id in order_df is a foreign key of id on customer_df.__

In [121]:
# Foreign keys for store_df

In [122]:
dataframes = [customer_df, product_df, product_subcategory_df,
           product_category_df, order_df, order_item_df]
store_foreign_keys_dict = {}  
  
for df in dataframes:  
    foreign_keys = find_foreign_keys(store_df, df)  
    store_foreign_keys_dict[id(df)] = foreign_keys  
  
print(store_foreign_keys_dict)

{2276199300080: [('country', 'country')], 2276209177952: [], 2276226530432: [], 2276199232576: [], 2276235311104: [], 2276230857216: []}


__Store_id on order_df is a foreign key for id on store_df.__

In [123]:
# Foreign keys for product_df

In [124]:
dataframes = [customer_df, store_df, product_subcategory_df,
           product_category_df, order_df, order_item_df]
prod_foreign_keys_dict = {}  
  
for df in dataframes:  
    foreign_keys = find_foreign_keys(product_df, df)  
    prod_foreign_keys_dict[id(df)] = foreign_keys  
  
print(prod_foreign_keys_dict)

{2276199300080: [], 2276207450720: [], 2276226530432: [], 2276199232576: [], 2276235311104: [], 2276230857216: [('product_cost', 'cost'), ('product_price', 'price')]}


__product_id on order_item_df is a foreign key for id on produt_df.__

In [125]:
# Foreign keys for product_subcategory_df

In [126]:
dataframes = [customer_df, store_df, product_df,
           product_category_df, order_df, order_item_df]
prod_subcat_foreign_keys_dict = {}  
  
for df in dataframes:  
    foreign_keys = find_foreign_keys(product_subcategory_df, df)  
    prod_subcat_foreign_keys_dict[id(df)] = foreign_keys  
  
print(prod_subcat_foreign_keys_dict)

{2276199300080: [], 2276207450720: [], 2276209177952: [], 2276199232576: [], 2276235311104: [], 2276230857216: [('line_item', 'category_id')]}


__category_id on product_subcategory_df is a foreign key of id on product_category_df__

In [127]:
# Foreign keys for order_df

In [128]:
dataframes = [customer_df, store_df, product_df, product_subcategory_df,
           product_category_df, order_item_df]
order_foreign_keys_dict = {}  
  
for df in dataframes:  
    foreign_keys = find_foreign_keys(order_df, df)  
    order_foreign_keys_dict[id(df)] = foreign_keys  
  
print(prod_foreign_keys_dict)

{2276199300080: [], 2276207450720: [], 2276226530432: [], 2276199232576: [], 2276235311104: [], 2276230857216: [('product_cost', 'cost'), ('product_price', 'price')]}


__order_id on order_item_df is a foreign key for order_df.__

### Create a test database

In [130]:
import pyodbc  
  
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=OLUWATOBI;"
    "Trusted_Connection=yes;"
    "DATABASE=master"
)

# Create a cursor object  
cursor = conn.cursor()  
  
# Create the database  
database_name = 'test_OneStopdB'
cursor.execute("CREATE DATABASE " + database_name)  
  
# Commit the transaction  
conn.commit()  
  
# Close the cursor and connection  
cursor.close()  
conn.close()

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Database 'test_OneStopdB' already exists. Choose a different database name. (1801) (SQLExecDirectW)")

### Write dfs into Database

In [132]:
#server = 'OLUWATOBI'   Use your db server name
database = 'test_OneStopdB'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver.replace(" ", "+")}'
engine = create_engine(connection_string)

In [133]:
# Write the DataFrame to a SQL table
customer_df.to_sql('customer', engine, if_exists='replace', index=False)

97

In [134]:
# Query the table to verify the data
query = "SELECT top 10 * FROM customer"
df_from_sql = pd.read_sql(query, engine)

# Print the DataFrame retrieved from SQL Server
df_from_sql

,gender,name,city,state,zip,country,continent,dob
0,Male,Tyler Vaught,London,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,Male,Frank Upchurch,Auberry,California,93602,United States,North America,1995-11-20
2,Female,Joan Roche,Red Deer,Alberta,T4N 2A6,Canada,North America,1962-08-24
3,Male,Sam Nelson,Burscough Bridge,West Lancashire,L40 8UX,United Kingdom,Europe,1979-11-28
4,Male,Ouassim Heuver,Leeuwarden,Friesland,8916 CD,Netherlands,Europe,1995-06-22
5,Female,Damiana Corona,Winchester,Virginia,22601,United States,North America,1944-10-30
6,Male,Callisto Lo Duca,Casilli,Napoli,80047,Italy,Europe,1976-08-28
7,Female,Hazel Skiles,Peterborough,Ontario,K9H 2L1,Canada,North America,1988-06-10
8,Female,Carlota Etzel,Sainte-Gertrude,Quebec,G0X 2S0,Canada,North America,1971-02-02
9,Male,Mike Mogensen,Chicago,Illinois,60605,United States,North America,1937-06-10


In [135]:
# Write the DataFrame to a SQL table
store_df.to_sql('store', engine, if_exists='replace', index=False)

58

In [136]:
# Write the DataFrame to a SQL table
product_df.to_sql('product', engine, if_exists='replace', index=False)

49

In [137]:
# Write the DataFrame to a SQL table
product_subcategory_df.to_sql('product_subcategory', engine, if_exists='replace', index=False)

32

In [138]:
# Write the DataFrame to a SQL table
product_category_df.to_sql('product_category', engine, if_exists='replace', index=False)

8

In [139]:
# Write the DataFrame to a SQL table
order_df.to_sql('order', engine, if_exists='replace', index=False)

126

In [140]:
# Write the DataFrame to a SQL table
order_item_df.to_sql('order_item', engine, if_exists='replace', index=False)

34

### Create the database

In [179]:
import pyodbc  
  
conn = pyodbc.connect(
    "Driver={SQL Server};"
    #"Server=OLUWATOBI;" Use your server name
    "Trusted_Connection=yes;"
    "DATABASE=master"
)

# Create a cursor object  
cursor = conn.cursor()  
  
# Create the database  
database_name = 'OneStopdB'
cursor.execute("CREATE DATABASE " + database_name)  
  
# Commit the transaction  
conn.commit()  
  
# Close the cursor and connection  
cursor.close()  
conn.close()

In [180]:
# Close the SQL engine
engine.dispose()

### Create Tables

In [141]:
#server = 'OLUWATOBI'   Use your db server name
database = 'OneStopdB'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver.replace(" ", "+")}'
engine = create_engine(connection_string)

In [142]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, BigInteger 
from sqlalchemy.orm import declarative_base, sessionmaker  

Base = declarative_base()  
  
class Customer(Base):  
    __tablename__ = 'customer'  
    id = Column(BigInteger, primary_key=True)  
    gender = Column(String(50))  
    name = Column(String(250))  
    city = Column(String(250))  
    state = Column(String(250))  
    zip = Column(String(10))  
    country = Column(String(250))  
    continent = Column(String(250))  
    dob = Column(DateTime)  
  
Base.metadata.create_all(engine)

In [143]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float, ForeignKey, BigInteger
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

class Order(Base):  
        __tablename__ = 'order'  
        id = Column(BigInteger, primary_key=True)  
        customer_id = Column(BigInteger, ForeignKey('customer.id'))  
        store_id = Column(BigInteger, ForeignKey('store.id'))  
        order_date = Column(DateTime)  
        delivery_date = Column(DateTime)  
  
        customer = relationship("Customer", backref="orders")
        store = relationship("Store", backref="orders")
  
class OrderItem(Base):  
        __tablename__ = 'order_item'  
        order_id = Column(BigInteger, primary_key=True)  
        line_item = Column(BigInteger)  
        product_id = Column(BigInteger, ForeignKey('product.id'))  
        quantity = Column(BigInteger)  
        product_cost = Column(Float)  
        product_price = Column(Float)  
  
        product = relationship("Product", backref="order_items")  
  
class Product(Base):  
        __tablename__ = 'product'  
        id = Column(BigInteger, primary_key=True)  
        name = Column(String(250))  
        brand = Column(String(250))  
        color = Column(String(250))  
        cost = Column(Float)  
        price = Column(Float)  
        subcategory_id = Column(BigInteger, ForeignKey('product_subcategory.id'))  
  
        subcategory = relationship("ProductSubcategory", backref="products")  
  
class ProductCategory(Base):  
        __tablename__ = 'product_category'  
        id = Column(Integer, primary_key=True)  
        category = Column(String(250))  
  
class Store(Base):  
        __tablename__ = 'store'  
        id = Column(BigInteger, primary_key=True)  
        country = Column(String(250))  
        state = Column(String(250))  
        sq_meters = Column(Integer)  
        open_date = Column(DateTime)  
  
class ProductSubcategory(Base):  
        __tablename__ = 'product_subcategory'  
        id = Column(BigInteger, primary_key=True)  
        category = Column(String(250))  
        category_id = Column(Integer, ForeignKey('product_category.id'))  
  
        category = relationship("ProductCategory", backref="subcategories")  
  
Base.metadata.create_all(engine)

In [158]:
# Close the SQL engine
engine.dispose()

__Moving over to SQL Server__

<div style="text-align: center;">
    <img src = "https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExZjMyYXJ5dG1ocDlpejA2NWdiamFvYmNlbTV1OXF5eHMyNGM3YjEzZCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/Y08EyNI2cZu5mo0C1k/giphy.webp" alt="Cookie" style="width: 600px; height: 400px;" />
</div>